In [24]:
import time
import re
import json
import statistics as stats
import requests as req
import numpy as np
import pandas as pd
from unidecode import unidecode
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import tqdm
import feedparser
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [25]:
# Configurar opciones de Chrome
chrome_options = Options()

chrome_options.add_argument(r"user-data-dir=C:\\Users\\User\\AppData\\Local\\Google\\Chrome\\User Data")
chrome_options.add_argument("--remote-allow-origins=*")
chrome_options.add_experimental_option("useAutomationExtension", False) 
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_argument("user-data-dir=selenium-cookies")
chrome_options.add_argument('--headless')  # Habilitar si no queremos ver la ventana
chrome_options.add_experimental_option("detach", True)  # Esta opción corrige el error de cierre repentino
chrome_options.add_argument('--start-minimized')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-infobars')


In [20]:
# Intenta abrir el navegador
try:
    driver = webdriver.Chrome(options=chrome_options)
    print("Navegador abierto correctamente")
except Exception as e:
    print(f"Error al abrir el navegador: {e}")
    exit()  # Sale del script si no se pudo abrir el navegador


Navegador abierto correctamente


In [26]:
# Otras importaciones y configuraciones de Selenium
driver = webdriver.Chrome(options=chrome_options)


In [27]:
def obtener_hrefs(driver, xpath):
    # Espera implícita para asegurarse de que la página esté completamente cargada
    driver.implicitly_wait(10)
    
    # Espera explícita para asegurarse de que los elementos estén presentes antes de buscarlos
    wait = WebDriverWait(driver, 10)
    elementos = wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath)))
    
    hrefs = []
    
    for elemento in elementos:
        try:
            # Obtén el enlace (href) dentro del elemento
            href = elemento.find_element(By.XPATH, './/a').get_attribute('href')
            hrefs.append(href)
        except NoSuchElementException:
            # Manejo de excepción si no se encuentra el elemento href
            print("No se encontró el elemento href en:", elemento.text)
    
    return hrefs

# Ejemplo de uso
url = 'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1'
xpath_elementos = '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li'

# Obtener hrefs
hrefs = obtener_hrefs(driver, xpath_elementos)

# Imprimir los hrefs obtenidos
print(hrefs)


TimeoutException: Message: 


In [12]:
def obtener_hrefs(driver, xpath):
    # Espera implícita para asegurarse de que la página esté completamente cargada
    driver.implicitly_wait(10)
    
    # Espera explícita para asegurarse de que los elementos estén presentes antes de buscarlos
    wait = WebDriverWait(driver, 10)
    elementos = wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath)))
    
    hrefs = []
    
    for elemento in elementos:
        try:
            # Obtén el enlace (href) dentro del elemento
            href = elemento.find_element(By.XPATH, './/a').get_attribute('href')
            hrefs.append(href)
        except NoSuchElementException:
            # Manejo de excepción si no se encuentra el elemento href
            print("No se encontró el elemento href en:", elemento.text)
    
    return hrefs

# Ejemplo de uso
url = 'tu_url_aqui'

tipos_objetos = ['Armas', 'Alimentos', 'Otros']  # Puedes agregar más tipos según sea necesario
xpath_base = '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[{}]/div/p[1]/a'

diccionario_xpaths = {tipo: xpath_base.format(i) for i, tipo in enumerate(tipos_objetos, start=1)}

for tipo, xpath in diccionario_xpaths.items():
    print(f'Hrefs para {tipo}:')
    hrefs = obtener_hrefs(driver, xpath)
    print(hrefs)
    print()


Hrefs para Armas:


TimeoutException: Message: 


In [ ]:
for href in hrefs:
    print(href)

In [14]:

def obtener_hrefs(driver, xpath):
    # Espera implícita para asegurarse de que la página esté completamente cargada
    driver.implicitly_wait(10)
    
    # Espera explícita para asegurarse de que los elementos estén presentes antes de buscarlos
    wait = WebDriverWait(driver, 20)  # Aumenta el tiempo de espera a 20 segundos
    try:
        elementos = wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath)))
    except TimeoutException:
        print(f'Tiempo de espera agotado para el XPath: {xpath}')
        return []

    hrefs = []
    
    for elemento in elementos:
        try:
            # Obtén el enlace (href) dentro del elemento
            href = elemento.find_element(By.XPATH, './/a').get_attribute('href')
            hrefs.append(href)
        except NoSuchElementException:
            # Manejo de excepción si no se encuentra el elemento href
            print("No se encontró el elemento href en:", elemento.text)
    
    return hrefs

# Ejemplo de uso
url = 'https://www.museodelprado.es/coleccion/obras-de-arte?'

# Construir diccionario de xpaths con un bucle
xpath_base = '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[{}]/div/p[1]/a'
diccionario_xpaths = {}

for i in range(1, 41):  # Cambia el rango según la cantidad de elementos que tengas
    tipo = f'tipo{i}'
    xpath = xpath_base.format(i)
    diccionario_xpaths[tipo] = xpath

# Imprimir el diccionario para verificar
print(diccionario_xpaths)


{'tipo1': '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[1]/div/p[1]/a', 'tipo2': '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[2]/div/p[1]/a', 'tipo3': '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[3]/div/p[1]/a', 'tipo4': '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[4]/div/p[1]/a', 'tipo5': '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[5]/div/p[1]/a', 'tipo6': '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[6]/div/p[1]/a', 'tipo7': '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[7]/div/p[1]/a', 'tipo8': '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[8]/div/p[1]/a', 'tipo9': '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[9]/div/p[1]/a', 'tipo10': '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[

In [12]:
def generar_xpaths(base_xpath, inicio, fin):
    return [base_xpath.format(i) for i in range(inicio, fin + 1)]

# Ejemplo de uso
base_xpath = '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[{}]/div/p[1]/a'
inicio = 1
fin = 40

xpaths_generados = generar_xpaths(base_xpath, inicio, fin)

for xpath in xpaths_generados:
    print(xpath)

//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[1]/div/p[1]/a
//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[2]/div/p[1]/a
//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[3]/div/p[1]/a
//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[4]/div/p[1]/a
//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[5]/div/p[1]/a
//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[6]/div/p[1]/a
//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[7]/div/p[1]/a
//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[8]/div/p[1]/a
//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[9]/div/p[1]/a
//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[10]/div/p[1]/a
//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[11]/div/p[1]/a
//*[@id=

In [22]:
def obtener_hrefs(driver, xpaths):
    # Espera implícita para asegurarse de que la página esté completamente cargada
    driver.implicitly_wait(10)
    
    hrefs_totales = []

    for xpath in xpaths:
        # Espera explícita para asegurarse de que los elementos estén presentes antes de buscarlos
        wait = WebDriverWait(driver, 10)
        elementos = wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath)))
        
        for elemento in elementos:
            try:
                # Obtén el enlace (href) dentro del elemento
                href = elemento.find_element(By.XPATH, './/a').get_attribute('href')
                hrefs_totales.append(href)
            except NoSuchElementException:
                # Manejo de excepción si no se encuentra el elemento href
                print("No se encontró el elemento href en:", elemento.text)

    return hrefs_totales

# Ejemplo de uso
url = 'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1'
base_xpath = '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[{}]/div/p[1]/a'
inicio = 1
fin = 40

xpaths_generados = generar_xpaths(base_xpath, inicio, fin)

# Obtener hrefs
hrefs_totales = obtener_hrefs(driver, xpaths_generados)

# Imprimir los hrefs obtenidos
print(hrefs_totales)


NameError: name 'generar_xpaths' is not defined

In [23]:
def obtener_hrefs(driver, xpaths):
    # Espera implícita para asegurarse de que la página esté completamente cargada
    driver.implicitly_wait(10)
    
    hrefs_totales = []

    for xpath in xpaths:
        print(f'Buscando elementos con el XPath: {xpath}')
        
        # Espera explícita para asegurarse de que la página esté completamente cargada
        wait_page_load = WebDriverWait(driver, 20)
        try:
            wait_page_load.until(EC.presence_of_element_located((By.XPATH, '/Resultados"]/*[@id="pan')))
        except TimeoutException:
            print('Tiempo de espera agotado para cargar la página')
            continue

        # Espera explícita para asegurarse de que los elementos estén presentes antes de buscarlos
        wait = WebDriverWait(driver, 10)
        try:
            elementos = wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath)))
        except TimeoutException:
            print(f'Tiempo de espera agotado para el XPath: {xpath}')
            continue

        for elemento in elementos:
            try:
                # Obtén el enlace (href) dentro del elemento
                href = elemento.find_element(By.XPATH, './/a').get_attribute('href')
                hrefs_totales.append(href)
            except NoSuchElementException:
                # Manejo de excepción si no se encuentra el elemento href
                print("No se encontró el elemento href en:", elemento.text)

    return hrefs_totales

# Ejemplo de uso
url = 'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1'
base_xpath = '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[{}]/div/p[1]/a'
inicio = 1
fin = 40

xpaths_generados = generar_xpaths(base_xpath, inicio, fin)

# Obtener hrefs
hrefs = obtener_hrefs(driver, xpaths_generados)

# Imprimir los hrefs obtenidos
print(hrefs)

NameError: name 'generar_xpaths' is not defined